In [1]:
import os
import gym
from stable_baselines3 import A2C # the algorithm itself
from stable_baselines3.common.vec_env import VecFrameStack # to vectorize environments (parallel training)
from stable_baselines3.common.evaluation import evaluate_policy # to evaluate the scores
from stable_baselines3.common.env_util import make_atari_env

/home/heisenberg/anaconda3/envs/RL/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

In [3]:
torch.cuda.is_available()

True

In [5]:
#!pip install atari_py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 9.3 MB/s eta 0:00:000m eta 0:00:0136m0:00:01


In [8]:
#!python -m atari_py.import_roms ./ROMS/ROMS

### Load and Visualize the Environment

In [4]:
env_name = "Breakout-v0"
env = gym.make(env_name)

A.L.E: Arcade Learning Environment (version 0.7.4+069f8bd)
[Powered by Stella]


In [5]:
env.reset()

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [6]:
env.action_space

Discrete(4)

In [7]:
env.observation_space

Box([[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]], [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
 

In [8]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
    print(f"Episode {episode} ended with score {score}")

env.close()

/home/heisenberg/anaconda3/envs/RL/lib/python3.9/site-packages/gym/envs/atari/environment.py:267: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  logger.warn(


Episode 1 ended with score 0.0
Episode 2 ended with score 3.0
Episode 3 ended with score 0.0
Episode 4 ended with score 1.0
Episode 5 ended with score 2.0


### Vectorize Environment & Train Model with Parallel Games(speeds up)

In [21]:
env = make_atari_env('Breakout-v0' , n_envs = 2, seed = 0) # this v0 uses images
env = VecFrameStack(env, n_stack = 2)

In [22]:
log_path = os.path.join('./train-artifacts', 'logs')
model = A2C('CnnPolicy', env, verbose = 1, tensorboard_log = log_path)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [25]:
a2c_path = os.path.join('./train-artifacts','saved_models','A2C_Breakout_v2')

In [26]:
model.learn(total_timesteps=1000000)
model.save(a2c_path)

Logging to ./train-artifacts/logs/A2C_2
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 281      |
|    ep_rew_mean        | 1.59     |
| time/                 |          |
|    fps                | 152      |
|    iterations         | 100      |
|    time_elapsed       | 6        |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -1.38    |
|    explained_variance | 0.00654  |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 0.563    |
|    value_loss         | 0.491    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 274      |
|    ep_rew_mean        | 1.46     |
| time/                 |          |
|    fps                | 156      |
|    iterations         | 200      |
|    time_elapsed       | 12       |
|    total_timesteps    | 2000     

In [27]:
model = A2C.load(a2c_path)

### Evaluate & Test

In [33]:
env_eval = make_atari_env('Breakout-v0', n_envs = 1, seed = 0, )
env_eval = VecFrameStack(env_eval, n_stack = 2, )
# evaluate_policy(model, env_eval, n_eval_episodes=10, render=True)
evaluate_policy(model, env_eval, n_eval_episodes=10, render=True)



/home/heisenberg/anaconda3/envs/RL/lib/python3.9/site-packages/gym/envs/atari/environment.py:267: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  logger.warn(


(12.1, 3.176476034853718)

In [34]:
env_eval.close()

In [32]:
evaluate_policy(model, env_eval, n_eval_episodes=10, render=False)


(15.6, 4.5431266766402185)